In [ ]:
!pip install torch-1.8.0+cu101-cp36-cp36m-linux_x86_64.whl
!pip install torch-geometric
!pip install torch_spline_conv-1.2.1-cp36-cp36m-linux_x86_64.whl
!pip install torch_scatter-2.0.8-cp36-cp36m-linux_x86_64.whl
!pip install torch_cluster-1.5.9-cp36-cp36m-linux_x86_64.whl
!pip install torch_sparse-0.6.10-cp36-cp36m-linux_x86_64.whl

In [ ]:
!pip install ogb

In [ ]:
#torch_geometric
#torch_sparse
#torch_scatter
#ogb


In [ ]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
import random
from models import GCN

import torch_geometric

import torch_sparse

import matplotlib.pyplot as plt
from sklearn import metrics

from utils import get_plot


In [2]:
from data_process import generate_data, load_data
from train_func import test, train, Lhop_Block_matrix_train

In [3]:
def get_K_hop_neighbors(adj_matrix, index, K):
    adj_matrix = adj_matrix + torch.eye(adj_matrix.shape[0],adj_matrix.shape[1])  #make sure the diagonal part >= 1
    hop_neightbor_index=index
    for i in range(K):
        hop_neightbor_index=torch.unique(torch.nonzero(adj[hop_neightbor_index])[:,1])
    return hop_neightbor_index

In [4]:
import scipy.sparse as sp
def normalize(mx):  #adj matrix
    
    mx = mx + torch.eye(mx.shape[0],mx.shape[1])
    
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return torch.tensor(mx)

# Model

In [5]:
#define model

#for compare 2-10 layer performance in appendix
#iterations = 400
#Adam, lr = 0.01


def centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers):
    
        model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        model.reset_parameters()
        if args_cuda:
                #from torch_geometric.nn import DataParallel
                #model = DataParallel(model)
                #model= torch.nn.DataParallel(model)
                model=model.cuda()
                
                #features= torch.nn.DataParallel(features)
                
                features = features.cuda()
                
                #edge_index= torch.nn.DataParallel(edge_index)
                
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        
        optimizer = optim.SGD(model.parameters(),
                              lr=args_lr, weight_decay=args_weight_decay)
        
        
        #optimizer = optim.Adam(model.parameters(),
        #                      lr=args_lr, weight_decay=args_weight_decay)
        # Train model
        best_val=0
        for t in range(args_iterations): #make to equivalent to federated
            loss_train, acc_train=train(t, model, optimizer, features, adj, labels, idx_train)
            # validation
            loss_train, acc_train= test(model, features, adj, labels, idx_train) #train after backward
            print(t,"train",loss_train,acc_train)
            loss_val, acc_val= test(model, features, adj, labels, idx_val) #validation
            print(t,"val",loss_val,acc_val)
            
            a = open(dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations),'a+')
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            
        #test  
        loss_test, acc_test= test(model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        
        print("save file as",dataset_name+'_IID_'+'centralized_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations))
        del model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        
        return loss_test, acc_test

In [6]:
def setdiff1d(t1, t2):
    
    combined = torch.cat((t1, t2))
    uniques, counts = combined.unique(return_counts=True)
    difference = uniques[counts == 1]
    #intersection = uniques[counts > 1]
    return difference
    

In [7]:
def intersect1d(t1, t2):
    
    combined = torch.cat((t1, t2))
    uniques, counts = combined.unique(return_counts=True)
    #difference = uniques[counts == 1]
    intersection = uniques[counts > 1]
    return intersection

In [8]:
def BDS_federated_GCN(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, sample_rate =0.5, L_hop=1, num_layers=2):
        # K: number of models
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].cuda()
                global_model=global_model.cuda()
                features = features.cuda()
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).cuda(), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []

        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).cuda()
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)   
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix

            
            
        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    diff = setdiff1d(split_data_indexes[i], communicate_indexes[i])
                    sample_index = torch.cat((split_data_indexes[i], diff[torch.randperm(len(diff))[:int(len(diff) * sample_rate)]])).clone()

                    sample_index = sample_index.sort()[0]
                    
                    inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)
                    in_sample_train_data_index = torch.searchsorted(sample_index, inter).clone()   #local id in block matrix

                    if len(in_sample_train_data_index) == 0:
                        continue
                    try:
                        adj[sample_index][:,sample_index]
                    except: #adj is empty
                        continue
                    
                    acc_train = FedSage_train(epoch, models[i], optimizers[i], 
                                                        features, adj, labels, sample_index, in_sample_train_data_index)
                acc_trains.append(acc_train)
                
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_BDS_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [ ]:
def FedSage_plus(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, L_hop = 1, num_layers=2):
        # K: number of models
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].to(device)
                global_model=global_model.to(device)
                features = features.to(device)
                adj = adj.to(device)
                labels = labels.to(device)
                idx_train = idx_train.to(device)
                idx_val = idx_val.to(device)
                idx_test = idx_test.to(device)
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).to(device), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []
        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).to(device)
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
                
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)

                
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix
        
        features_in_clients = []
        #assume the linear generator learnt the optimal (the average of features of neighbor nodes)
        #gaussian noise
        for i in range(K):
            #orignial features of outside neighbors of nodes in client i
            original_feature_i = features[setdiff1d(split_data_indexes[i], communicate_indexes[i])].clone()
            
            gaussian_feature_i = original_feature_i + torch.normal(0, 0.1, original_feature_i.shape).to(device)
            
            copy_feature = features.clone()
            
            copy_feature[setdiff1d(split_data_indexes[i], communicate_indexes[i])] = gaussian_feature_i
            
            features_in_clients.append(copy_feature[communicate_indexes[i]])
            print(features_in_clients[i].shape, communicate_indexes[i].shape)

        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    if len(in_com_train_data_indexes[i]) == 0:
                        continue
                    try:
                        adj[communicate_indexes[i]][:,communicate_indexes[i]]
                    except: #adj is empty
                        continue
                    acc_train = FedSage_train(epoch, models[i], optimizers[i], 
                                                        features_in_clients[i], adj, labels, communicate_indexes[i], in_com_train_data_indexes[i])
                    
                acc_trains.append(acc_train)
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_FedSage_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [9]:
def Lhop_Block_federated_GCN(K, features, adj, labels, idx_train, idx_val, idx_test, iid_percent, L_hop, num_layers):
        # K: number of models
        #choose adj matrix
        #multilayer_GCN:n*n
        #define model
        global_model = GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers)
        global_model.reset_parameters()
        models=[]
        for i in range(K):
            models.append(GCN(nfeat=features.shape[1],
                nhid=args_hidden,
                nclass=labels.max().item() + 1,
                dropout=args_dropout,
                NumLayers=num_layers))
        if args_cuda:
                for i in range(K):
                    models[i]=models[i].cuda()
                global_model=global_model.cuda()
                features = features.cuda()
                adj = adj.cuda()
                labels = labels.cuda()
                idx_train = idx_train.cuda()
                idx_val = idx_val.cuda()
                idx_test = idx_test.cuda()
        #optimizer and train
        optimizers=[]
        for i in range(K):
            optimizers.append(optim.SGD(models[i].parameters(),
                              lr=args_lr, weight_decay=args_weight_decay))
        # Train model
        
        row, col, edge_attr = adj.t().coo()
        edge_index = torch.stack([row, col], dim=0)
        
        
        split_data_indexes=[]
        
        nclass=labels.max().item() + 1
        split_data_indexes = []
        non_iid_percent = 1 - float(iid_percent)
        iid_indexes = [] #random assign
        shuffle_labels = [] #make train data points split into different devices
        for i in range(K):
            current = torch.nonzero(labels == i).reshape(-1)
            current = current[np.random.permutation(len(current))] #shuffle
            shuffle_labels.append(current)
                
        average_device_of_class = K // nclass
        if K % nclass != 0: #for non-iid
            average_device_of_class += 1
        for i in range(K):  
            label_i= i // average_device_of_class    
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * non_iid_percent)
            split_data_indexes.append((labels_class[average_num * (i % average_device_of_class):average_num * (i % average_device_of_class + 1)]))
        
        if args_cuda:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))).cuda(), torch.cat(split_data_indexes))
        else:
            iid_indexes = setdiff1d(torch.tensor(range(len(labels))), torch.cat(split_data_indexes))
        
        
        for i in range(K):  #for iid
            label_i= i // average_device_of_class
            labels_class = shuffle_labels[label_i]

            average_num= int(len(labels_class)//average_device_of_class * (1 - non_iid_percent))
            split_data_indexes[i] = list(split_data_indexes[i]) + list(iid_indexes[:average_num])
                    
            iid_indexes = iid_indexes[average_num:]
            
        communicate_indexes = []
        in_com_train_data_indexes = []
        for i in range(K):
            if args_cuda:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i]).cuda()
            else:
                split_data_indexes[i] = torch.tensor(split_data_indexes[i])
                
            split_data_indexes[i] = split_data_indexes[i].sort()[0]
            
            #communicate_index=get_K_hop_neighbors(adj, split_data_indexes[i], L_hop) #normalized adj
            
            communicate_index = torch_geometric.utils.k_hop_subgraph(split_data_indexes[i],L_hop,edge_index)[0]
                
            communicate_indexes.append(communicate_index)
            communicate_indexes[i] = communicate_indexes[i].sort()[0]
            
            inter = intersect1d(split_data_indexes[i], idx_train)  ###only count the train data of nodes in current server(not communicate nodes)

                
            in_com_train_data_indexes.append(torch.searchsorted(communicate_indexes[i], inter).clone())   #local id in block matrix

        #assign global model weights to local models at initial step
        for i in range(K):
            models[i].load_state_dict(global_model.state_dict())
        
        for t in range(args_iterations):
            acc_trains=[]
            for i in range(K):
                for epoch in range(args_epochs):
                    if len(in_com_train_data_indexes[i]) == 0:
                        continue
                    
                    try:
                        adj[communicate_indexes[i]][:,communicate_indexes[i]]
                    except: #adj is empty
                        continue
                    acc_train = Lhop_Block_matrix_train(epoch, models[i], optimizers[i], 
                                                        features, adj, labels, communicate_indexes[i], in_com_train_data_indexes[i])
                    
                acc_trains.append(acc_train)
            states=[]
            gloabl_state=dict()
            for i in range(K):
                states.append(models[i].state_dict())
            # Average all parameters
            for key in global_model.state_dict():
                gloabl_state[key] = in_com_train_data_indexes[0].shape[0] * states[0][key]
                count_D=in_com_train_data_indexes[0].shape[0]
                for i in range(1,K):
                    gloabl_state[key] += in_com_train_data_indexes[i].shape[0] * states[i][key]
                    count_D += in_com_train_data_indexes[i].shape[0]
                gloabl_state[key] /= count_D

            global_model.load_state_dict(gloabl_state)
            
            # Testing
            
            loss_train, acc_train = test(global_model, features, adj, labels, idx_train)
            print(t,'\t',"train",'\t',loss_train,'\t',acc_train)
            
            loss_val, acc_val = test(global_model, features, adj, labels, idx_val) #validation
            print(t,'\t',"val",'\t',loss_val,'\t',acc_val)
            

            a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')

            
            a.write(str(t)+'\t'+"train"+'\t'+str(loss_train)+'\t'+str(acc_train)+'\n')
            a.write(str(t)+'\t'+"val"+'\t'+str(loss_val)+'\t'+str(acc_val)+'\n')
            a.close()
            for i in range(K):
                models[i].load_state_dict(gloabl_state)
        #test  
        loss_test, acc_test= test(global_model, features, adj, labels, idx_test)
        print(t,'\t',"test",'\t',loss_test,'\t',acc_test)
        a = open(dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K),'a+')
        a.write(str(t)+'\t'+"test"+'\t'+str(loss_test)+'\t'+str(acc_test)+'\n')
        a.close()
        print("save file as",dataset_name+'_IID_'+str(iid_percent)+'_' + str(L_hop) +'hop_Block_federated_' + str(num_layers) + 'layer_GCN_iter_'+str(args_iterations)+'_epoch_'+str(args_epochs)+'_device_num_'+str(K))
        
        del global_model
        del features 
        del adj
        del labels
        del idx_train
        del idx_val
        del idx_test
        while(len(models)>=1):
            del models[0]
        
        return loss_test, acc_test



In [10]:
np.random.seed(42)
torch.manual_seed(42)
#'cora', 'citeseer', 'pubmed' #simulate #other dataset twitter, 
dataset_name="ogbn-arxiv"#'ogbn-arxiv'

if dataset_name == 'simulate':
    number_of_nodes=200
    class_num=3
    link_inclass_prob=10/number_of_nodes  #when calculation , remove the link in itself
    #EGCN good when network is dense 20/number_of_nodes  #fails when network is sparse. 20/number_of_nodes/5

    link_outclass_prob=link_inclass_prob/20


    features, adj, labels, idx_train, idx_val, idx_test = generate_data(number_of_nodes,  class_num, link_inclass_prob, link_outclass_prob)               
else:
    features, adj, labels, idx_train, idx_val, idx_test = load_data(dataset_name)
    class_num = labels.max().item() + 1




In [11]:
if dataset_name in ['simulate', 'cora', 'citeseer', 'pubmed']:
    args_hidden = 16
else:
    args_hidden = 256

args_dropout = 0.5
args_lr = 1.0
args_weight_decay = 5e-4     #L2 penalty
args_epochs = 3
args_no_cuda = False
args_cuda = not args_no_cuda and torch.cuda.is_available()

args_device_num = class_num #split data into args_device_num parts
args_iterations = 500

In [12]:
#for i in range(10):
#    centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)

In [ ]:
for i in range(10):
    for args_epochs in [3]:
        for args_random_assign in [1.0, 0.5, 0.0]:
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = 2)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = 2)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = 2)
            

0 	 train 	 3.172605037689209 	 0.2541757842997108
0 	 val 	 3.1301751136779785 	 0.24809557367696902
1 	 train 	 3.1332666873931885 	 0.1790611495365127
1 	 val 	 3.1789023876190186 	 0.07627772744051814
2 	 train 	 3.013021469116211 	 0.24135428464608924
2 	 val 	 2.984691619873047 	 0.21118158327460654
3 	 train 	 2.9503469467163086 	 0.18175520392342287
3 	 val 	 2.9643988609313965 	 0.07906305580724185
4 	 train 	 3.2560362815856934 	 0.10993941126664541
4 	 val 	 2.9522557258605957 	 0.22973925299506695
5 	 train 	 2.8682262897491455 	 0.18616465620567182
5 	 val 	 2.9188342094421387 	 0.08443236350213094
6 	 train 	 3.3049325942993164 	 0.11015933407374012
6 	 val 	 2.9400179386138916 	 0.22987348568743918
7 	 train 	 2.853689193725586 	 0.3814561089057741
7 	 val 	 2.826092004776001 	 0.38880499345615627
8 	 train 	 2.769745349884033 	 0.23491054639821424
8 	 val 	 2.8200979232788086 	 0.19832880297996577
9 	 train 	 2.999749183654785 	 0.1227938993413312
9 	 val 	 2.7291998863

80 	 train 	 1.772853970527649 	 0.5327190156255155
80 	 val 	 1.7885456085205078 	 0.5738783180643646
81 	 train 	 1.7903891801834106 	 0.5522261686148162
81 	 val 	 1.7882084846496582 	 0.5124668613040706
82 	 train 	 1.729055404663086 	 0.556976501248062
82 	 val 	 1.7390350103378296 	 0.5910265445149167
83 	 train 	 1.7312462329864502 	 0.5751971058158587
83 	 val 	 1.7359265089035034 	 0.5454881036276386
84 	 train 	 1.7059770822525024 	 0.5628264479167813
84 	 val 	 1.7115085124969482 	 0.593576965669989
85 	 train 	 1.7241390943527222 	 0.5818607668708283
85 	 val 	 1.7205028533935547 	 0.5625356555589114
86 	 train 	 1.703334927558899 	 0.5552611033527232
86 	 val 	 1.7052804231643677 	 0.576529413738716
87 	 train 	 1.7349390983581543 	 0.5860173079249184
87 	 val 	 1.719037413597107 	 0.5893150776871707
88 	 train 	 1.7128349542617798 	 0.5360068615915814
88 	 val 	 1.7121399641036987 	 0.5307560656397866
89 	 train 	 1.7424237728118896 	 0.5895470689787885
89 	 val 	 1.73346

159 	 train 	 1.5691733360290527 	 0.5746692910788314
159 	 val 	 1.5792316198349 	 0.5613275613275613
160 	 train 	 1.5740575790405273 	 0.6177741612693944
160 	 val 	 1.5784788131713867 	 0.6258263700124165
161 	 train 	 1.5758405923843384 	 0.568236548971311
161 	 val 	 1.5719172954559326 	 0.557602604114232
162 	 train 	 1.5742335319519043 	 0.6200063777614058
162 	 val 	 1.5967471599578857 	 0.6192154099130843
163 	 train 	 1.5711660385131836 	 0.572733970376398
163 	 val 	 1.5619267225265503 	 0.5670660089264741
164 	 train 	 1.5598022937774658 	 0.6202922774106289
164 	 val 	 1.5890038013458252 	 0.6135105204872647
165 	 train 	 1.5552211999893188 	 0.5831363191519777
165 	 val 	 1.5492507219314575 	 0.5785764622973926
166 	 train 	 1.544734001159668 	 0.6218207409199371
166 	 val 	 1.572558045387268 	 0.6171348031813149
167 	 train 	 1.5453826189041138 	 0.5881065745923181
167 	 val 	 1.5413707494735718 	 0.5830396993187691
168 	 train 	 1.5363632440567017 	 0.6218317370602918


237 	 train 	 1.4939700365066528 	 0.5978161665255496
237 	 val 	 1.5106229782104492 	 0.588308332494379
238 	 train 	 1.4963809251785278 	 0.6211499763582983
238 	 val 	 1.4921528100967407 	 0.6227054599147622
239 	 train 	 1.491898536682129 	 0.6004662363510408
239 	 val 	 1.5089699029922485 	 0.595355548843921
240 	 train 	 1.4968523979187012 	 0.61899473284877
240 	 val 	 1.4923166036605835 	 0.6139803349105675
241 	 train 	 1.489867091178894 	 0.602412553193829
241 	 val 	 1.505863070487976 	 0.603610859424813
242 	 train 	 1.497658371925354 	 0.6178621303922324
242 	 val 	 1.4953275918960571 	 0.6085439108694922
243 	 train 	 1.488257884979248 	 0.6036111324924952
243 	 val 	 1.5011531114578247 	 0.6098191214470284
244 	 train 	 1.4956761598587036 	 0.6176971882869113
244 	 val 	 1.4964401721954346 	 0.6050874190409075
245 	 train 	 1.4878783226013184 	 0.6023685686324101
245 	 val 	 1.4975422620773315 	 0.6122688680828215
246 	 train 	 1.4948737621307373 	 0.6184889103924522
246

315 	 train 	 1.472162127494812 	 0.5961887377530487
315 	 val 	 1.4723563194274902 	 0.6017987180777878
316 	 train 	 1.4802922010421753 	 0.6235471349556306
316 	 val 	 1.4906706809997559 	 0.6049196281754421
317 	 train 	 1.4760459661483765 	 0.5969584675778802
317 	 val 	 1.475936770439148 	 0.6127386825061244
318 	 train 	 1.4757640361785889 	 0.6215678296917782
318 	 val 	 1.4861340522766113 	 0.6004899493271586
319 	 train 	 1.4621973037719727 	 0.6045238121419382
319 	 val 	 1.468471646308899 	 0.6162287325078023
320 	 train 	 1.4645346403121948 	 0.6239319998680464
320 	 val 	 1.4707427024841309 	 0.60720158394577
321 	 train 	 1.4580186605453491 	 0.6075587468798452
321 	 val 	 1.4689221382141113 	 0.6141481257760327
322 	 train 	 1.4629534482955933 	 0.6239649882891105
322 	 val 	 1.4631133079528809 	 0.6127722406792174
323 	 train 	 1.4574449062347412 	 0.6073718124938147
323 	 val 	 1.473393201828003 	 0.606664653176281
324 	 train 	 1.462205410003662 	 0.6239539921487558


...

222 	 train 	 1.6205519437789917 	 0.5805632223089696
222 	 val 	 1.599349856376648 	 0.5986778079801336
223 	 train 	 1.6203150749206543 	 0.5806401952914527
223 	 val 	 1.599588394165039 	 0.5984093425953891
224 	 train 	 1.6186041831970215 	 0.5804752531861317
224 	 val 	 1.5977219343185425 	 0.5981408772106447
225 	 train 	 1.617964744567871 	 0.5811790061688348
225 	 val 	 1.5971060991287231 	 0.5985435752877614
226 	 train 	 1.617314338684082 	 0.5814868980987673
226 	 val 	 1.5964939594268799 	 0.5995838786536462
227 	 train 	 1.6155503988265991 	 0.5817178170462168
227 	 val 	 1.594974160194397 	 0.5990805060572503
228 	 train 	 1.6152034997940063 	 0.5821906510814704
228 	 val 	 1.594861626625061 	 0.5998859022114836
229 	 train 	 1.6139203310012817 	 0.5823555931867914
229 	 val 	 1.5933585166931152 	 0.5988120406725058
230 	 train 	 1.6120556592941284 	 0.5823885816078557
230 	 val 	 1.5915669202804565 	 0.5998523440383906
231 	 train 	 1.6120699644088745 	 0.582597508274595

300 	 train 	 1.5586358308792114 	 0.59426441319097
300 	 val 	 1.5411436557769775 	 0.6099533541394007
301 	 train 	 1.5573914051055908 	 0.5945283205594837
301 	 val 	 1.5400502681732178 	 0.610792308466727
302 	 train 	 1.5563746690750122 	 0.5946162896823215
302 	 val 	 1.5389612913131714 	 0.6104231685627034
303 	 train 	 1.5570323467254639 	 0.5947262510858689
303 	 val 	 1.5400127172470093 	 0.6110607738514715
304 	 train 	 1.5545202493667603 	 0.5952760581036056
304 	 val 	 1.536847472190857 	 0.6106245176012618
305 	 train 	 1.5557478666305542 	 0.5950671314368656
305 	 val 	 1.5383923053741455 	 0.6105574012550756
306 	 train 	 1.553981065750122 	 0.5950781275772205
306 	 val 	 1.5366443395614624 	 0.6105238430819826
307 	 train 	 1.553802728652954 	 0.5951990851211225
307 	 val 	 1.5366485118865967 	 0.6110943320245646
308 	 train 	 1.5538560152053833 	 0.5953970156475077
308 	 val 	 1.5364971160888672 	 0.6105909594281688
309 	 train 	 1.5531017780303955 	 0.595572953893183

378 	 train 	 1.5201656818389893 	 0.6027644296851805
378 	 val 	 1.5056689977645874 	 0.6174368267391523
379 	 train 	 1.5199016332626343 	 0.6027534335448258
379 	 val 	 1.505400538444519 	 0.6169334541427565
380 	 train 	 1.5192607641220093 	 0.6023685686324101
380 	 val 	 1.5046025514602661 	 0.6161951743347093
381 	 train 	 1.5186958312988281 	 0.6026984528430521
381 	 val 	 1.5042604207992554 	 0.6170341286620357
382 	 train 	 1.5178940296173096 	 0.6030613254747583
382 	 val 	 1.5033689737319946 	 0.6168998959696634
383 	 train 	 1.5177949666976929 	 0.6029073795097921
383 	 val 	 1.503589391708374 	 0.6168998959696634
384 	 train 	 1.517369270324707 	 0.6033032405625626
384 	 val 	 1.5029960870742798 	 0.6175710594315246
385 	 train 	 1.5165812969207764 	 0.6034461903871741
385 	 val 	 1.5022485256195068 	 0.6179401993355482
386 	 train 	 1.5173941850662231 	 0.603160290737951
386 	 val 	 1.5029243230819702 	 0.6168663377965704
387 	 train 	 1.5165501832962036 	 0.6029513640712

456 	 train 	 1.494165301322937 	 0.6076357198623283
456 	 val 	 1.4821536540985107 	 0.6209268767408302
457 	 train 	 1.4939510822296143 	 0.6076247237219736
457 	 val 	 1.4817795753479004 	 0.6203228296251552
458 	 train 	 1.4943766593933105 	 0.6076357198623283
458 	 val 	 1.4821972846984863 	 0.6204906204906205
459 	 train 	 1.4925891160964966 	 0.6076577121430378
459 	 val 	 1.4803593158721924 	 0.620759085875365
460 	 train 	 1.4931401014328003 	 0.6078666388097778
460 	 val 	 1.4809222221374512 	 0.6208597603946441
461 	 train 	 1.4923725128173828 	 0.6077566774062304
461 	 val 	 1.480047583580017 	 0.6210946676062955
462 	 train 	 1.4931303262710571 	 0.607316831792041
462 	 val 	 1.4805597066879272 	 0.6202557132789691
463 	 train 	 1.4922348260879517 	 0.607734685125521
463 	 val 	 1.480076789855957 	 0.6207255277022719
464 	 train 	 1.4926530122756958 	 0.607899627230842
464 	 val 	 1.4804977178573608 	 0.6209604349139233
465 	 train 	 1.4918197393417358 	 0.6081195500379367

34 	 train 	 2.312187910079956 	 0.33121474362498765
34 	 val 	 2.2497141361236572 	 0.32618544246451225
35 	 train 	 2.2972116470336914 	 0.3366578331005817
35 	 val 	 2.2342560291290283 	 0.32910500352360816


In [ ]:
BDS_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign)

In [ ]:

for i in range(10):
    centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = 2)
    
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for i in range(10):
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 0, num_layers = num_l)
            BDS_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = num_l)
            Lhop_Block_federated_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = num_l)

In [ ]:
for num_l in range(1, 11):
    for i in range(10):#10 times
        centralized_GCN(features, adj, labels, idx_train, idx_val, idx_test, num_layers = num_l)

In [ ]:
for args_epochs in [3]:
    for args_random_assign in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for num_l in range(2, 11):
            for i in range(10):
                Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, num_layers = num_l)
                Lhop_Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 1, num_layers = num_l)
                Lhop_Block_federated_multilayer_GCN(class_num, features, adj, labels, idx_train, idx_val, idx_test, args_random_assign, 2, num_layers = num_l)